In [1]:
import os
sep_local = os.path.sep
sep_local

'\\'

In [2]:
%env TF_KERAS = 1

env: TF_KERAS=1


In [3]:
import sys
sys.path.append('..'+sep_local+'..')

In [4]:
#done in case of linux
#import os
#os.chdir("/home/azeghost/git/Generative_Models")

In [5]:
# from tensorflow.keras.mixed_precision import experimental as mixed_precision
# policy = mixed_precision.Policy('mixed_float16')
# mixed_precision.set_policy(policy)
# print('Compute dtype: %s' % policy.compute_dtype)
# print('Variable dtype: %s' % policy.variable_dtype)

In [6]:
import tensorflow as tf

In [7]:
tf.__version__

'2.0.0-rc1'

In [8]:
dataset_name = 'mnist'

In [9]:
import tensorflow_datasets as tfds

info = tfds.builder(dataset_name).info

print(info)

tfds.core.DatasetInfo(
    name='mnist',
    version=1.0.0,
    description='The MNIST database of handwritten digits.',
    urls=['https://storage.googleapis.com/cvdf-datasets/mnist/'],
    features=FeaturesDict({
        'image': Image(shape=(28, 28, 1), dtype=tf.uint8),
        'label': ClassLabel(shape=(), dtype=tf.int64, num_classes=10),
    }),
    total_num_examples=70000,
    splits={
        'test': 10000,
        'train': 60000,
    },
    supervised_keys=('image', 'label'),
    citation="""@article{lecun2010mnist,
      title={MNIST handwritten digit database},
      author={LeCun, Yann and Cortes, Corinna and Burges, CJ},
      journal={ATT Labs [Online]. Available: http://yann. lecun. com/exdb/mnist},
      volume={2},
      year={2010}
    }""",
    redistribution_info=,
)



In [10]:
intermediate_dim = 20
inputs_shape=(28, 28, 1) # image shape
batch_size = 100
latent_dim = 50

In [11]:
TRAIN_BUF = 600
TEST_BUF = 100


In [12]:
import tensorflow_datasets as tfds
# Construct a tf.data.Dataset
train_ds = tfds.load(name=dataset_name, split=tfds.Split.TRAIN).shuffle(TRAIN_BUF).batch(batch_size)
try:
    test_ds = tfds.load(name=dataset_name, split=tfds.Split.TEST).shuffle(TEST_BUF).batch(batch_size)
except:
    test_ds = tfds.load(name=dataset_name, split=tfds.Split.TRAIN).shuffle(TEST_BUF).batch(batch_size)


In [13]:
_instance_scale=1.0
for data in train_ds:
    _instance_scale = float(data['image'][0].numpy().max())
    break

In [14]:
_instance_scale

255.0

In [15]:
enc_lays2 = [
    tf.keras.layers.Conv2D(filters=32, kernel_size=3, strides=(2, 2), activation='relu'),
    tf.keras.layers.Conv2D(filters=64, kernel_size=3, strides=(2, 2), activation='relu'),
    tf.keras.layers.Flatten(),
    # No activation
    tf.keras.layers.Dense(latent_dim, dtype='float32')
]

dec_lays2 = [
    tf.keras.layers.Dense(units=7*7*32, activation=tf.nn.relu),
    tf.keras.layers.Reshape(target_shape=(7, 7, 32)),
    tf.keras.layers.Conv2DTranspose(filters=64, kernel_size=3, strides=(2, 2), padding="SAME", activation='relu'),
    tf.keras.layers.Conv2DTranspose(filters=32, kernel_size=3, strides=(2, 2), padding="SAME", activation='relu'),
    
    # No activation
    tf.keras.layers.Conv2DTranspose(filters=1, kernel_size=3, strides=(1, 1), padding="SAME", dtype='float32')
]

In [16]:
enc_mean_lays = [tf.keras.layers.Dense(units=intermediate_dim, activation='relu'),
            tf.keras.layers.Dense(units=intermediate_dim, activation='relu'),
            tf.keras.layers.Dense(units=intermediate_dim, activation='relu')]

enc_var_lays = [tf.keras.layers.Dense(units=intermediate_dim, activation='relu'),
            tf.keras.layers.Dense(units=intermediate_dim, activation='relu'),
            tf.keras.layers.Dense(units=intermediate_dim, activation='relu')]

dec_lays = [tf.keras.layers.Dense(units=2*intermediate_dim, activation='relu'),
            tf.keras.layers.Dense(units=2*intermediate_dim, activation='relu'),
            tf.keras.layers.Dense(units=2*intermediate_dim, activation='relu')]

## First Time

In [17]:
#from utils.data_and_files.file_utils import make_random_string
#from time import gmtime, strftime

#model_name = 'AE_' + make_random_string(5) + strftime("%a_%d_%b_%Y_%H_%M", gmtime())
#print(model_name)

In [18]:
model_name = dataset_name+'AE'

In [19]:
recoding_dir='..'+sep_local+'..'+sep_local+'recoding'+sep_local+model_name

In [20]:
import os

In [21]:
os.path.join(recoding_dir, model_name)

'..\\..\\recoding\\mnistAE\\mnistAE'

In [22]:
pwd

'C:\\projects\\Generative_Models\\notebooks\\MNIST'

In [23]:
from training.traditional.autoencoders.autoencoder import autoencoder as AE

DEBUG:matplotlib.pyplot:Loaded backend module://ipykernel.pylab.backend_inline version unknown.
DEBUG:matplotlib.pyplot:Loaded backend module://ipykernel.pylab.backend_inline version unknown.


In [24]:
variables_params = \
[
    {
        'name': 'inference', 
        'inputs_shape':inputs_shape,
        'outputs_shape':latent_dim,
        'layers': enc_lays2#enc_mean_lays
    }

    ,
    
        {
        'name': 'generative', 
        'inputs_shape':latent_dim,
        'outputs_shape':inputs_shape,
        'layers':dec_lays2#dec_lays
    }
]

In [25]:
from utils.data_and_files.file_utils import create_if_not_exist

In [26]:
_restore = os.path.join(recoding_dir, 'var_save_dir')

In [27]:
create_if_not_exist(_restore)
_restore

'..\\..\\recoding\\mnistAE\\var_save_dir'

In [28]:
#to restore trained model, set filepath=_restore

In [29]:
ae = AE( 
    name=model_name,
    inputs_shape=inputs_shape,
    outputs_shape=inputs_shape,
    latent_dim=latent_dim,
    variables_params=variables_params, 
    filepath=None
    )

Model: "inference"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 13, 13, 32)        320       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 6, 6, 64)          18496     
_________________________________________________________________
flatten (Flatten)            (None, 2304)              0         
_________________________________________________________________
dense (Dense)                (None, 50)                115250    
_________________________________________________________________
batch_normalization (BatchNo (None, 50)                200       
_________________________________________________________________
dropout (Dropout)            (None, 50)                0         
_________________________________________________________________
activity_regularization (Act (None, 50)                0 

  WARNING  | None


Model: "generative"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 1568)              79968     
_________________________________________________________________
reshape (Reshape)            (None, 7, 7, 32)          0         
_________________________________________________________________
conv2d_transpose (Conv2DTran (None, 14, 14, 64)        18496     
_________________________________________________________________
conv2d_transpose_1 (Conv2DTr (None, 28, 28, 32)        18464     
_________________________________________________________________
conv2d_transpose_2 (Conv2DTr (None, 28, 28, 1)         289       
_________________________________________________________________
batch_normalization_1 (Batch (None, 28, 28, 1)         4         
_________________________________________________________________
dropout_1 (Dropout)          (None, 28, 28, 1)         0

  WARNING  | None


In [30]:
ae.compile()

Instructions for updating:
Use `tf.cast` instead.


Instructions for updating:
Use `tf.cast` instead.


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Model: "mnistAE"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inference_inputs (InputLayer [(None, 28, 28, 1)]       0         
_________________________________________________________________
inference (Sequential)       (None, 50)                134266    
_________________________________________________________________
generative (Sequential)      (None, 28, 28, 1)         117221    
_________________________________________________________________
x_logits (TensorFlowOpLayer) [(None, 28, 28, 1)]       0         
Total params: 251,487
Trainable params: 251,385
Non-trainable params: 102
_________________________________________________________________
None


In [31]:
import numpy as np

In [32]:
from keras.datasets import mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.astype(np.float32)/255.0
x_test = x_test.astype(np.float32)/255.0

Using TensorFlow backend.


In [33]:
from keras_tqdm import TQDMNotebookCallback

In [34]:
TQDMNotebookCallback?

In [35]:
progbar_cb1 = TQDMNotebookCallback(leave_outer=False)
progbar_cb2 = tf.keras.callbacks.ProgbarLogger(count_mode='samples', stateful_metrics='loss')
setattr(progbar_cb1,'on_train_batch_begin',lambda x,y:None)

setattr(progbar_cb1,'on_train_batch_end',lambda x,y:None)
setattr(progbar_cb1,'on_test_batch_begin',lambda x,y:None)
setattr(progbar_cb1,'on_test_batch_end',lambda x,y:None)

setattr(progbar_cb1,'on_test_begin', lambda x:None)
setattr(progbar_cb1,'on_test_end', lambda x:None)

In [36]:
es = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', 
    min_delta=1e-12, 
    patience=5, 
    verbose=1, 
    restore_best_weights=True
)

In [37]:
class ModelSaver(tf.keras.callbacks.Callback):
    def __init__(self, filepath, save_freq=5, **kws):
        self.save_freq = save_freq
        self.filepath = filepath
        tf.keras.callbacks.Callback.__init__(self, **kws)
        
    def on_epoch_end(self, epoch, logs={}):
        if epoch%self.save_freq==0:  # or save after some epoch, each k-th epoch etc.
            self.model.save(self.filepath)

In [38]:
ms = ModelSaver(filepath=_restore,save_freq=5)

In [39]:
csv_dir = os.path.join(recoding_dir, 'csv_dir')
create_if_not_exist(csv_dir)
csv_dir = os.path.join(csv_dir, ae.name+'.csv')
csv_log = tf.keras.callbacks.CSVLogger(csv_dir, append=True)
csv_dir

'..\\..\\recoding\\mnistAE\\csv_dir\\mnistAE.csv'

In [40]:
from utils.reporting.ploting import plot_and_save_generated
from IPython import display
class SampleGeneration(tf.keras.callbacks.Callback):
    def __init__(
        self, 
        filepath, 
        gen_freq=5, 
        save_img=False, 
        random_latent=None, 
        latent_shape=50, 
        gray_plot=False,
        **kws
    ):
        self.gen_freq = gen_freq
        self.filepath = filepath
        self.save_img = save_img
        self.gray_plot = gray_plot
        
        if random_latent is None:
            self.random_latent = tf.random.normal(shape=[50, latent_shape])
        else:
            self.random_latent = random_latent
            
        tf.keras.callbacks.Callback.__init__(self, **kws)
        
    def on_epoch_end(self, epoch, logs={}):
        if epoch%self.gen_freq==0:  # or save after some epoch, each k-th epoch etc.
            generated = self.model.generate_sample(model=self.model.get_varibale, 
                                                   inputs_shape=self.model.inputs_shape, 
                                                   latent_shape=[50, self.model.latent_dim], 
                                                   eps=self.random_latent)
            plot_and_save_generated(generated=generated, epoch=epoch, path=self.filepath, gray=self.gray_plot, save=self.save_img)
            

In [41]:
image_gen_dir = os.path.join(recoding_dir, 'image_gen_dir')
create_if_not_exist(image_gen_dir)

In [42]:
sg = SampleGeneration(filepath=image_gen_dir, gen_freq=5, save_img=True, gray_plot=True)

In [43]:
#ae.fit?

In [44]:
progbar_cb2 = tf.keras.callbacks.ProgbarLogger(count_mode='samples', stateful_metrics=None)

In [45]:
ae.fit(epochs=100, 
       x=x_train.reshape((x_train.shape + (1,))),
       y=x_train.reshape((x_train.shape + (1,))), 
       validation_freq=1,
       validation_data=(x_test.reshape((x_test.shape + (1,))),
                        x_test.reshape((x_test.shape + (1,)))),
       verbose=0,
       callbacks=[progbar_cb1, es, ms, csv_log, sg])

Restoring model weights from the end of the best epoch.


Epoch 00031: early stopping


In [ ]:
@####

In [ ]:
import numpy as np
n_generated = 50
random_latent = np.random.normal(size=[n_generated, latent_dim])
#np.save(file='random_latent.npy', arr=random_latent)
#random_latent = tf.constant(np.load(file='..\\data\\random_latent.npy'))

In [ ]:
ae.fit(
    train_dataset=train_ds, 
    test_dataset=test_ds,
    instance_names=['image'],
    epochs=10,
    learning_rate=1e-3,
    random_latent=random_latent,
    recoding_dir=recoding_dir,
    gray_plot=True,
    generate_epoch=5,
    save_epoch=5,
    metric_epoch=10,
    gt_data=None
)

In [ ]:
#from ground_truth_datasets.datasets import load

In [ ]:
for batch in train_ds:
    image = batch['image'].numpy()[0]
    break

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.imshow(image.reshape(28,28))

In [ ]:
reconst = ae.decode(ae.encode(image))

In [ ]:
plt.imshow(reconst.numpy().reshape(28,28))